In [1]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from tensorflow.keras.layers import Dense , Dropout
from scikeras.wrappers import KerasRegressor 
from tensorflow.keras.callbacks import EarlyStopping
import time
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import GridSearchCV

In [2]:
#data = pd.read_pickle('C:/Users/erikm/Desktop/Diplomarbeit Erik Marr/Daten/Finish/TPath_300_finish_data.pkl')
data = pd.read_pickle('C:/Users/erikm/Desktop/Diplomarbeit Erik Marr/Daten/Finish/Finish_D4_t_I7000_F9000/D4_t.pkl')

data  

,X-Koordinate,Y-Koordinate,Zeitpunkt,Strom,Kraft,Temperatur
0,0.0000,-0.00200,100,7000,9000,400.35
1,0.0000,-0.00192,100,7000,9000,416.89
2,0.0000,-0.00184,100,7000,9000,432.05
3,0.0000,-0.00176,100,7000,9000,445.87
4,0.0000,-0.00168,100,7000,9000,458.33
...,...,...,...,...,...,...
43906,0.0024,0.00168,500,7000,9000,775.40
43907,0.0024,0.00176,500,7000,9000,715.43
43908,0.0024,0.00184,500,7000,9000,645.85
43909,0.0024,0.00192,500,7000,9000,585.87


In [3]:
df = data.drop(data.columns[3:5], axis = 1)
df

,X-Koordinate,Y-Koordinate,Zeitpunkt,Temperatur
0,0.0000,-0.00200,100,400.35
1,0.0000,-0.00192,100,416.89
2,0.0000,-0.00184,100,432.05
3,0.0000,-0.00176,100,445.87
4,0.0000,-0.00168,100,458.33
...,...,...,...,...
43906,0.0024,0.00168,500,775.40
43907,0.0024,0.00176,500,715.43
43908,0.0024,0.00184,500,645.85
43909,0.0024,0.00192,500,585.87


In [4]:
df1 = df.sample(frac=1, random_state=42)  # Hier wird 42 als Random State verwendet, um die Ergebnisse reproduzierbar zu machen

print(df1)
df_reset = df1.reset_index(drop=True)
df_reset

       X-Koordinate  Y-Koordinate  Zeitpunkt  Temperatur
38620       0.00012      -0.00096        460     1229.80
14428       0.00108       0.00168        230      542.76
14359       0.00096       0.00024        230      777.04
21104       0.00168       0.00128        290      720.23
28698       0.00192       0.00088        360      886.28
...             ...           ...        ...         ...
6265        0.00204       0.00144        150      539.86
11284       0.00132      -0.00096        200      690.53
38158       0.00156      -0.00120        450     1049.80
860         0.00192       0.00152        100      464.69
15795       0.00180       0.00088        240      715.22

[43911 rows x 4 columns]


,X-Koordinate,Y-Koordinate,Zeitpunkt,Temperatur
0,0.00012,-0.00096,460,1229.80
1,0.00108,0.00168,230,542.76
2,0.00096,0.00024,230,777.04
3,0.00168,0.00128,290,720.23
4,0.00192,0.00088,360,886.28
...,...,...,...,...
43906,0.00204,0.00144,150,539.86
43907,0.00132,-0.00096,200,690.53
43908,0.00156,-0.00120,450,1049.80
43909,0.00192,0.00152,100,464.69


In [5]:
label = df_reset["Temperatur"]
# Korrektur: Verwenden Sie den Spaltennamen direkt, ohne Indexierung der columns-Eigenschaft
df1 = df_reset.drop("Temperatur", axis=1)
X = df1
y = label


In [6]:
X

,X-Koordinate,Y-Koordinate,Zeitpunkt
0,0.00012,-0.00096,460
1,0.00108,0.00168,230
2,0.00096,0.00024,230
3,0.00168,0.00128,290
4,0.00192,0.00088,360
...,...,...,...
43906,0.00204,0.00144,150
43907,0.00132,-0.00096,200
43908,0.00156,-0.00120,450
43909,0.00192,0.00152,100


In [7]:
y

0        1229.80
1         542.76
2         777.04
3         720.23
4         886.28
          ...   
43906     539.86
43907     690.53
43908    1049.80
43909     464.69
43910     715.22
Name: Temperatur, Length: 43911, dtype: float64

In [8]:
 # train_df enthält 80% der Daten, test_df enthält 20% der Daten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [9]:
# Initialisiere einen MinMaxScaler für die Features
scaler_features = MinMaxScaler()
scaler_features2 = MinMaxScaler()
# Skaliere X_train und X_test
X_train_scaled = scaler_features.fit_transform(X_train)
X_test_scaled = scaler_features.transform(X_test)  # Nutze unterschiedliche Skalierungsparameter

# Initialisiere einen SEPARATEN MinMaxScaler für das Ziel, wenn nötig
scaler_target = MinMaxScaler()


# Skaliere y_train und y_test. Beachte, dass y_train.reshape(-1, 1) verwendet wird, da MinMaxScaler 
# erwartet, dass die Eingaben als 2D-Arrays kommen, und Ziele normalerweise als 1D-Arrays vorliegen.
y_train_scaled = scaler_target.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_target.transform(y_test.values.reshape(-1, 1))


In [10]:
X_train_scaled

array([[0.05 , 0.66 , 0.275],
       [0.8  , 0.02 , 0.525],
       [0.6  , 0.2  , 0.525],
       ...,
       [0.45 , 0.92 , 0.975],
       [0.3  , 0.7  , 0.875],
       [0.75 , 1.   , 0.275]])

In [11]:
y_train_scaled.max()

0.9999999999999999

In [13]:
# Netzwerkarchitektur
model = Sequential([

    Dense(232, activation='relu', input_shape=(3,), kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),
    
    Dense(152, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),
    
    Dense(232, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),
    
    Dense(1 , activation = 'linear')
])

# Optimierer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

# Modell kompilieren (Verwendung von mean_squared_error als Verlustfunktion für Regression)
model.compile(optimizer=optimizer,
              loss='mean_squared_error',
              metrics=['mae'])  # Metriken für Regression: Mean Absolute Error und Mean Squared Error

# Early Stopping Callback
early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1, mode='min', restore_best_weights=True)

# Trainingsparameter
batch_size = 16
epochs = 2000

# Modell trainieren (Annahme: X_train, y_train, X_val, y_val sind vordefiniert)
history = model.fit(X_train_scaled, y_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2,
                    callbacks=[early_stopping])

Epoch 1/2000
1757/1757 [==============================] - 3s 1ms/step - loss: 0.0316 - mae: 0.0913 - val_loss: 0.0186 - val_mae: 0.0618
Epoch 2/2000
1757/1757 [==============================] - 2s 1ms/step - loss: 0.0174 - mae: 0.0581 - val_loss: 0.0164 - val_mae: 0.0533
Epoch 3/2000
1757/1757 [==============================] - 3s 1ms/step - loss: 0.0155 - mae: 0.0516 - val_loss: 0.0148 - val_mae: 0.0491
Epoch 4/2000
1757/1757 [==============================] - 2s 1ms/step - loss: 0.0142 - mae: 0.0471 - val_loss: 0.0138 - val_mae: 0.0439
Epoch 5/2000
1757/1757 [==============================] - 3s 1ms/step - loss: 0.0133 - mae: 0.0433 - val_loss: 0.0128 - val_mae: 0.0420
Epoch 6/2000
1757/1757 [==============================] - 3s 2ms/step - loss: 0.0125 - mae: 0.0401 - val_loss: 0.0122 - val_mae: 0.0372
Epoch 7/2000
1757/1757 [==============================] - 3s 2ms/step - loss: 0.0119 - mae: 0.0373 - val_loss: 0.0116 - val_mae: 0.0361
Epoch 8/2000
1757/1757 [========================

KeyboardInterrupt: 

In [13]:
# # Initialisiere Listen, um Ergebnisse zu speichern
# val_loss_results = []
# val_mae_results = []
# 
# # Funktion, um das Modell zu erstellen
# def create_model():
#     model = Sequential([
#                 Dense(232, activation='relu', input_shape=(3,), kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),
#                 
#                 Dense(152, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),
#                 
#                 Dense(232, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.00001)),
#                 
#                 Dense(1 , activation = 'linear')
# 
#     ])
#     optimizer = Adam(learning_rate=0.00001)
#     model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
#     return model
# 
# # K-Fold Cross-Validation Konfiguration
# n_splits = 5
# kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
# 
# # Leistungsüberwachung
# fold_no = 1
# for train_index, val_index in kf.split(X_train_scaled):
#     X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
#     y_train_fold, y_val_fold = y_train_scaled[train_index], y_train_scaled[val_index]
# 
#     model = create_model()
# 
#     early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)
# 
#     print(f'Training für Fold {fold_no}...')
#     history = model.fit(X_train_fold, y_train_fold, batch_size=16, epochs=1000, validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping], verbose=1)
# 
#     # Speichere die Ergebnisse des aktuellen Folds
#     val_loss_results.append(min(history.history['val_loss']))
#     val_mae_results.append(min(history.history['val_mae']))
# 
#     fold_no += 1
# 
# # Berechne den Durchschnitt über alle Folds
# average_val_loss = np.mean(val_loss_results)
# average_val_mae = np.mean(val_mae_results)
# 
# # Gib die durchschnittlichen Ergebnisse aus
# print(f'Durchschnittlicher Validation Loss: {average_val_loss}')
# print(f'Durchschnittlicher Validation MAE: {average_val_mae}')


Training für Fold 1...
Epoch 1/1000
1757/1757 [==============================] - 3s 1ms/step - loss: 0.1269 - mae: 0.1721 - val_loss: 0.0220 - val_mae: 0.0781
Epoch 2/1000
1757/1757 [==============================] - 2s 1ms/step - loss: 0.0202 - mae: 0.0701 - val_loss: 0.0183 - val_mae: 0.0633
Epoch 3/1000
1757/1757 [==============================] - 2s 1ms/step - loss: 0.0173 - mae: 0.0597 - val_loss: 0.0162 - val_mae: 0.0569
Epoch 4/1000
1757/1757 [==============================] - 2s 1ms/step - loss: 0.0155 - mae: 0.0533 - val_loss: 0.0149 - val_mae: 0.0516
Epoch 5/1000
1757/1757 [==============================] - 2s 1ms/step - loss: 0.0144 - mae: 0.0490 - val_loss: 0.0139 - val_mae: 0.0480
Epoch 6/1000
1757/1757 [==============================] - 2s 1ms/step - loss: 0.0135 - mae: 0.0452 - val_loss: 0.0131 - val_mae: 0.0421
Epoch 7/1000
1757/1757 [==============================] - 2s 1ms/step - loss: 0.0128 - mae: 0.0417 - val_loss: 0.0125 - val_mae: 0.0415
Epoch 8/1000
1757/1757 [=

KeyboardInterrupt: 

In [14]:
results = model.evaluate(X_test_scaled, y_test_scaled, verbose=2)
results

275/275 - 0s - loss: 7.0876e-05 - mae: 0.0017 - 287ms/epoch - 1ms/step


[7.087632548063993e-05, 0.001725318725220859]

In [15]:
scaled_predicted_values = model.predict(X_test_scaled, verbose = 0)

# Führen Sie die Rücktransformation der skalierten Werte durch
original_predicted_values = scaler_target.inverse_transform(scaled_predicted_values)
original_actual_values = scaler_target.inverse_transform(y_test_scaled)  # y_test sind die skalierten tatsächlichen Werte
print(f' Bsp. Predicted: {original_predicted_values[100]} Actual: {original_actual_values[100]} ')

def calculate_mae(list1, list2):
    # Stelle sicher, dass beide Listen die gleiche Länge haben
    if len(list1) != len(list2):
        raise ValueError("Listen müssen die gleiche Länge haben")

    # Berechne die absolute Differenz zwischen den Elementen der Listen
    differences = [abs(x - y) for x, y in zip(list1, list2)]

    # Berechne den Durchschnitt der absoluten Differenzen
    mae = sum(differences) / len(differences)

    return mae

# Beispiel
list1 = original_predicted_values
list2 = original_actual_values

mae = calculate_mae(list1, list2)
print(f"Durchschnittliche Abweichung (MAE): {mae}")

 Bsp. Predicted: [598.49207] Actual: [596.74] 
Durchschnittliche Abweichung (MAE): [1.98123601]


In [16]:
df_result = pd.DataFrame({'Echt': [val[0] for val in list1], 'Vorhergesagt': [val[0] for val in list2]})
df_result['X-Koordinate'] = X_test_scaled[:, 0]
df_result['Y-Koordinate'] = X_test_scaled[:, 1]
df_result['Zeitpunkt'] = X_test_scaled[:, 2]

df_result['Differenz'] = df_result['Echt'] - df_result['Vorhergesagt']
df_result['Differenz'].sort_values()
sorted_df = df_result.sort_values(by= 'Differenz')
Anzahl_Punkte = (sorted_df['Differenz'] < -5).sum()
print("Anzahl der Werte die kleiner sind:", Anzahl_Punkte)

sorted_df


Anzahl der Werte die kleiner sind: 115


,Echt,Vorhergesagt,X-Koordinate,Y-Koordinate,Zeitpunkt,Differenz
5913,649.757568,673.72,1.00,0.94,0.700,-23.962432
1837,631.379456,648.25,1.00,0.94,0.625,-16.870544
1316,607.560913,623.39,1.00,0.94,0.550,-15.829087
6769,529.480530,543.98,1.00,0.94,0.225,-14.499470
655,815.239563,829.58,1.00,0.06,0.900,-14.340437
...,...,...,...,...,...,...
991,554.916138,538.83,0.95,0.98,0.750,16.086138
3222,413.788055,397.49,1.00,0.98,0.075,16.298055
573,426.371307,409.84,1.00,0.98,0.125,16.531307
7585,445.166748,428.07,1.00,0.98,0.200,17.096748


In [17]:
#Berechnung der Auswertungsgröße R^2

def calculate_r_squared(predicted, actual):
    # Berechnung des Mittelwerts der tatsächlichen Werte
    mean_actual = sum(actual) / len(actual)
    
    # Berechnung der totalen Summe der Quadrate (SST)
    sst = sum((x - mean_actual) ** 2 for x in actual)
    
    # Berechnung der Summe der Quadrate der Residuen (SSE)
    sse = sum((actual[i] - predicted[i]) ** 2 for i in range(len(actual)))
    
    # Berechnung des R^2-Wertes
    r_squared = 1 - (sse / sst)
    
    return r_squared

# Berechnung von R^2 mit den bereitgestellten Listen
r_squared = calculate_r_squared(list1, list2)

print(f"R^2: {r_squared}")

R^2: [0.99987614]


In [18]:
mae = history.history['loss']
val_mae = history.history['val_loss']

epochs = range(1, len(mae) + 1)

# MAE Diagramm
plt.figure(figsize=(10, 6))
plt.plot(epochs, mae, 'r', label='Training MSE')
plt.plot(epochs, val_mae, 'b', label='Validation MSE')
plt.title('Training and Validation MSE')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()

plt.show()

NameError: name 'history' is not defined

In [19]:
mae = history.history['mae']
val_mae = history.history['val_mae']

epochs = range(1, len(mae) + 1)

# MAE Diagramm
plt.figure(figsize=(10, 6))
plt.plot(epochs, mae, 'r', label='Training MAE')
plt.plot(epochs, val_mae, 'b', label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()


#plt.ylim(0.00, 0.01)


plt.show()

NameError: name 'history' is not defined

# GridSearch

In [12]:
def build_model(learning_rate=0.00001, activation='relu', regularization=0.00001, dropout_rate=0.0):
    model = Sequential()
    model.add(Dense(232, activation=activation, input_shape=(3,), kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))

    model.add(Dense(152, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))

    model.add(Dense(232, activation=activation, kernel_initializer='he_uniform', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['mae'])
    return model

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Verwenden Sie eine Funktion, um das Modell zu instanziieren, für scikit-learn Wrapper
model = KerasRegressor(model=build_model, verbose=2, callbacks=[early_stopping])

# Anpassung der Parameter im param_grid
param_grid = {
    'model__learning_rate': [0.00001],
    'model__regularization': [0.00001],
    'fit__batch_size': [16, 32, 64, 100],
    'fit__epochs': [100],
    'model__dropout_rate' : [0.0]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=2)
# Hinweis: Stellen Sie sicher, dass Ihre Daten (X_train_scaled, y_train_scaled) korrekt definiert sind
grid_result = grid_search.fit(X_train_scaled, y_train_scaled)
# Beste Parameter und Score ausgeben
print("Beste Parameter:", grid_search.best_params_)
print("Beste Genauigkeit:", grid_search.best_score_)

with open("Gridsearch_D4_t_1.txt", "w") as f:
    f.write(f"Beste Parameter: {grid_search.best_params_}\n")
    f.write(f"Beste Genauigkeit: {grid_search.best_score_}")


Fitting 3 folds for each of 4 candidates, totalling 12 fits


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001B625802E50>>
Traceback (most recent call last):
  File "C:\Users\erikm\Desktop\Diplomarbeit Erik Marr\Projekt X\venv\Lib\site-packages\ipykernel\ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
# # Funktion zum Erstellen des Modells
# def build_model(hp):
#     model = Sequential()
#     model.add(Dense(hp.Int('input_units', min_value=8, max_value=328, step=16), input_shape=(3,), activation='relu'))
#     for i in range(hp.Int('n_layers', 1, 10)):
#         model.add(Dense(hp.Int(f'units_{i}', min_value=8, max_value=328, step=16), activation='relu'))
#     model.add(Dense(1, activation='linear'))
#     model.compile(optimizer='adam', loss='mean_squared_error')
#     return model
# 
# # Durchführung der Random Search dreimal
# for run in range(1, 4):
#     # Anpassen des Verzeichnisses und des Projektnamens für jeden Durchlauf
#     directory = 'random_search'
#     project_name = f'random_search_D4_t_1_{run}'
# 
#     tuner = RandomSearch(
#         build_model,
#         objective='val_loss',
#         max_trials=100,
#         executions_per_trial=2,
#         directory=directory,
#         project_name=project_name
#     )
# 
#     # Durchführung des Random Search
#     tuner.search(X_train_scaled, y_train_scaled, epochs=50, verbose =0, batch_size=50, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])
# 
#     # Abrufen und Speichern des besten Modells
#     best_model = tuner.get_best_models(num_models=1)[0]
#     model_path = os.path.join(directory, project_name, 'best_model.h5') 
#     best_model.save(model_path)
# 
# 
#     # Optional: Abrufen und Ausgeben der besten Hyperparameter
#     best_hyperparameters = tuner.get_best_hyperparameters()[0]
# 
#     # Konvertieren der Hyperparameter in ein DataFrame
#     df_hyperparameters = pd.DataFrame([best_hyperparameters.values])
#     # Speichern des DataFrame als CSV
#     df_hyperparameters.to_csv(f'random_search_D4_t_1_{run}.csv', index=False)
# 
#     print(f"Beste Hyperparameter für Lauf {run}: {best_hyperparameters.values}")
